# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [41]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
from scipy.misc import derivative
@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [42]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

Ответ:
В этой реализации есть некоторые недостатки:
не поддерживаются унарные операции, такие как оператор минус, знак и т.д.
Оператор деления не поддерживается
Оператор возведения в степень не поддерживается

In [43]:
from __future__ import annotations
from scipy.misc import derivative
import math
import random
from itertools import repeat
from dataclasses import dataclass
from typing import Generic, List, Dict, Tuple, TypeVar, Union, Callable, Protocol

DefaultEps = 1e-5
Number = Union[int, float]

Number = Union[int, float]

@dataclass
class Dual:
    value: float
    d: float = 0.

    def __add__(self, other):
        return self._operate(other, lambda x, y: x + y)

    def __mul__(self, other):
        return self._operate(other, lambda x, y: x * y)

    __rmul__ = __mul__
    __radd__ = __add__

    def __repr__(self):
        return f'Dual<{self.value}, {self.d}>'

    def __neg__(self):
        return Dual(-self.value, -self.d)

    def __sub__(self, other):
        return self._operate(other, lambda x, y: x - y)

    def __rsub__(self, other):
        return self._operate(other, lambda x, y: y - x)

    def __truediv__(self, other):
        return self._operate(other, lambda x, y: x / y)

    def __pow__(self, other):
        return self._operate(other, lambda x, y: x ** y, lambda x, y: y * math.log(x) + x * y / x)

    def __rpow__(self, other):
        return self._operate(other, lambda x, y: y ** x, lambda x, y: y * math.log(x) + x * y / x)

    def _operate(self, other, operate, d_operate=None):
        if isinstance(other, Number):
            return Dual(operate(self.value, float(other)), self.d if d_operate is None else 0.)
        return Dual(operate(self.value, other.value), d_operate(self.value, self.d) if d_operate else self.d + other.d)


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [44]:
def exp(x: Union[Dual, Number]) -> Dual:
    if isinstance(x, Number):
        return Dual(math.exp(x))
    return Dual(math.exp(x.value), math.exp(x.value) * x.d)


def cos(x: Union[Dual, Number]) -> Dual:
    if isinstance(x, Number):
        return Dual(math.cos(x))
    return Dual(math.cos(x.value), -math.sin(x.value) * x.d)


def sin(x: Union[Dual, Number]) -> Dual:
    if isinstance(x, Number):
        return Dual(math.sin(x))
    return Dual(math.sin(x.value), math.cos(x.value) * x.d)


def log(x: Union[Dual, Number], base: float = math.e) -> Dual:
    if isinstance(x, Number):
        return Dual(math.log(x, base), 0)
    return Dual(math.log(x.value, base), x.d / (x.value * math.log(base)))

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [45]:
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [46]:
def derivative(func: Callable[[Dual], Dual], x: float, eps: float = DefaultEps) -> float:
    return (func(Dual(x + eps)).value - func(Dual(x - eps)).value) / (2 * eps)
def test(n: int, level: int = 3):
    funcs = [
        lambda x: x,
        lambda x: 3*x**2 + 2*x - 5,
        lambda x: 5 * x ** 2 + 2 * x + 2,
        lambda x: (((x+x)+(x+x))*((x*x)+(x+9))),
        lambda x: exp(x) + sin(2*x) - cos(3*x),
        lambda x: cos(x) * (x ** 2)
    ]
    for i in range(len(funcs)):
        func = funcs[i]
        print(f'func{i+1}', abs(derivative(func, 4) - diff(func)(4)))
        
# Tests
if __name__ == '__main__':
    test(n=100)

func1 1.565336749109747e-11
func2 24.999999999015696
func3 40.99999999983106
func4 252.99999999442025
func5 2.2917917053683823
func6 6.122888462764686


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [47]:
import random
from typing import Dict, Tuple, Callable

class RandomFunc:
    Functions: Dict[str, int] = {
        'cos': 1,
        'sin': 1,
        'exp': 1
    }

    Operators: Dict[str, int] = {
        '+': 2,
        '-': 2,
        '*': 2
    }

    def __init__(self, var: str, cmin: int = 1, cmax: int = 10) -> None:
        self.constant = lambda: str(random.randint(cmin, cmax))
        self.variable = lambda: var
        self.function_list = list(self.Functions.keys())
        self.operator_list = list(self.Operators.keys())

    @property
    def randleaf(self) -> str:
        if random.random() <= 0.8:
            return self.variable()
        return self.constant()

    @property
    def randfun(self) -> Tuple[str, int]:
        if random.random() <= 0.9:
            return random.choice(self.function_list),self.Functions[random.choice(self.function_list)]
        return random.choice(self.operator_list),self.Operators[random.choice(self.operator_list)]

    def g(self, maxlevel: int) -> str:
        if maxlevel == 0:
            return self.randleaf
        func, nparam = self.randfun
        params = [self.g(maxlevel - 1) for _ in range(nparam)]
        if nparam == 1:
            return f"{func}({params[0]})"
        else:
            return f"({params[0]}{func}{params[1]})"

    def __call__(self, maxlevel: int) -> str:
        return self.g(maxlevel)

def test(n: int, level: int = 3):
    generator = RandomFunc('x')
    for _ in range(n):
        expr = generator(maxlevel=level)
        func = eval(f'lambda x: {expr}')
        x = random.randint(0, 10) * random.random()
        try:
            numberal = derivative(func, x)
            auto = func(Dual(x, 1)).d
            ratio = abs(2 * (numberal - auto) / (numberal + auto))
        except:
            continue
        print(f'{expr} [{x}]: {ratio}')

if __name__ == '__main__':
    test(n=100)

sin(cos(sin(x))) [1.5956715818511764]: 1.8264299608829985e-10
sin(cos(sin(x))) [3.5375636860045927]: 1.7539199093444983e-11
exp(cos((x-1))) [0.848879720994575]: 7.022144293579563e-11
cos(cos(sin(x))) [0.6283265976327224]: 8.748929322096433e-11
sin(exp(exp(x))) [0.5389691805649832]: 2.093632096860842e-10
cos(exp(cos(x))) [3.1541015455074386]: 7.006358835295578e-10
cos(sin((6+x))) [5.180061619609388]: 9.537028245747539e-11
cos(exp((x-x))) [1.4895890055156418]: 2.0
exp(sin(cos(x))) [0.7245218848406304]: 2.8981625927298693e-11
cos(sin(exp(x))) [3.4201233620377844]: 5.95488560638733e-08
exp(sin(cos(x))) [0.025381145310208186]: 1.483148812511305e-10
exp(sin(sin(x))) [5.528833046617549]: 2.2647809091846477e-11
exp(exp(exp(x))) [0.3437398514922996]: 1.3744655590322167e-09
exp(cos(cos(x))) [1.6872292727906464]: 1.1293916210095663e-10
(cos(exp(x))+sin(sin(x))) [8.930998941545301]: 0.0009533275822827192
cos(cos(exp(x))) [5.488742406286562]: 2.619566918661177e-06
sin((sin(9)+exp(x))) [1.1007806403

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [48]:
from typing import TypeVar, Generic, List, Callable

T = TypeVar('T')

class Dual:
    def __init__(self, value: float, d: float):
        self.value = value
        self.d = d

class NCallable(Protocol, Generic[T]):

    def __call__(self, *args: T) -> T:
        pass

def identity(n: int) -> List[List[int]]:
    return [[(0, 1)[i == j] for i in range(n)] for j in range(n)]

def ndiff(func: NCallable[Dual]) -> Callable[..., List[float]]:
    
    def wrapper(*vars: float) -> List[float]:
        nvalue, nd = identity(len(vars)), identity(len(vars))
        result: List[float] = []
        for values, ds in zip(nvalue, nd):
            params: List[Dual] = []
            for value, d in zip(values, ds):
                params.append(Dual(value, d))
            result.append(func(*params).d)
        return result

    return wrapper

